In [1]:

library(caret, quiet=TRUE);
library(base64enc)
library(httr)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [5]:

## binary classification in iris dataset:
    
num_class <- 3
lb <- as.numeric(iris$Species) - 1

iris$Species2 = as.factor(sapply(iris$Species , function(x) if( x == 'setosa') 'yes' else 'no'))
dataset = as.matrix(iris[, 1:4])
    
create_model  =  function() {

    set.seed(1960)
    formula <- as.formula(Species2 ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width)
    model <- train(formula, iris, method = "glmnet", trace=FALSE)
    return(model)
}

In [6]:
head(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,Species2
5.1,3.5,1.4,0.2,setosa,yes
4.9,3.0,1.4,0.2,setosa,yes
4.7,3.2,1.3,0.2,setosa,yes
4.6,3.1,1.5,0.2,setosa,yes
5.0,3.6,1.4,0.2,setosa,yes
5.4,3.9,1.7,0.4,setosa,yes


In [7]:
# dataset
model = create_model()
# cat(model$feature_names)
# plot(model)


Warning message:
“model fit failed for Resample01: alpha=0.10, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample01: alpha=0.55, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample01: alpha=1.00, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample02: alpha=0.10, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample02: alpha=0.55, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample02: alpha=1.00, la

”Warning message:
“model fit failed for Resample15: alpha=1.00, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample16: alpha=0.10, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample16: alpha=0.55, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample16: alpha=1.00, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample17: alpha=0.10, lambda=0.087 Error in (function (x, y, family = c("gaussian", "binomial", "poisson",  : 
  unused argument (trace = FALSE)
”Warning message:
“model fit failed for Resample17: alpha=0.55, l

Something is wrong; all the Accuracy metric values are missing:
    Accuracy       Kappa    
 Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA  
 NA's   :9     NA's   :9    


ERROR: Error: Stopping


In [ ]:
pred <- predict(model, dataset , type="prob")
pred_labels <- predict(model, dataset , type="raw")
error_rate = sum(pred_labels != iris$Species2)/length(pred_labels)
error_rate

# SQL Code Generation

In [ ]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="xgboost+r")
    r = POST(WS_URL, body = data, encode = "json", verbose())
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [ ]:
# labels(model$finalModel)

In [ ]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)

# Execute the SQL Code

In [ ]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

In [ ]:
df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

In [ ]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [ ]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

In [ ]:
# colnames(df1)

# R XGBoost Output

In [ ]:
pred  =  predict(model, as.matrix(dataset), type = "prob")
# pred <- matrix(pred , ncol=3, byrow=TRUE)
# p0 = 1 - pred
df_r_out = data.frame(pred)
names(df_r_out) = sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(dataset))
df_r_out$Score_no  =  NA
df_r_out$Score_yes  =  NA
df_r_out$LogProba_no  =  log(df_r_out$Proba_no)
df_r_out$LogProba_yes  =  log(df_r_out$Proba_yes)
df_r_out$Decision =   predict(model, as.matrix(dataset), type = "raw")
df_r_out$DecisionProba =  apply(pred, 1, function(x) max(x))
head(df_r_out)


In [ ]:
# df_r_out

# Compare R and SQL output

In [ ]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

In [ ]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

In [ ]:
stopifnot(nrow(diffs_df) == 0)

In [ ]:
summary(df_sql_out)

In [ ]:
summary(df_r_out)